### ⚠ IMPORTANT ⚠

You will need at least 22GB of VRAM (GPU RAM) to run this notebook.

If you're running this locally - please ensure you have the correct hardware to support the fine-tuning.

Please make sure you're using the following instance:

![image](https://i.imgur.com/ji210Ug.png)

# Fine-tuning Embedding Models

In the following Notebook we will be exploring one of the most powerful techniques to take your single-domain RAG pipelines to the next level.

Fine-tuning Embeddings Models!

- 🤝 Breakout Room #2
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-l`
  - Task 5: Evaluating Retrieval with Embedding Model

But before any of that, we need to grab some dependencies, and set up some boilerplate!

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key, and Hugging Face token!

### Nest Asyncio

In [1]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [2]:
!pip install -qU llama-index-llms-openai llama-index-embeddings-openai llama-index-finetuning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -qU llama-index-readers-file llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.7 MB/s eta 0:00:00


In [4]:
!pip install -qU "sentence_transformers==2.7.0"

### API Key Section!

In classic fashion, we'll need to provide our OpenAI API key!

We'll also provide our Hugging Face token (with `Write` access) in order to save our model on the Hub!

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [6]:
from huggingface_hub import notebook_login

notebook_login()

## Task 2: Loading Data

The data can be found in [this GitHub repo](https://github.com/AI-Maker-Space/DataRepository/tree/main/high-performance-rag).

In this case, the data is related to research articles about Camelids (aka: Llamas, Alpacas, Camels!)

In [7]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 71 (delta 19), reused 28 (delta 8), pack-reused 8
Receiving objects: 100% (71/71), 69.00 MiB | 45.56 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [8]:
%cd ./DataRepository/high-performance-rag/

/content/DataRepository/high-performance-rag


In [9]:
!unzip "Camel Papers Test.zip"

Archive:  Camel Papers Test.zip
  inflating: Camel Papers Test/Acute respiratory distress syndrome in an alpaca cria.pdf  
  inflating: Camel Papers Test/Alpaca liveweight variations and fiber production in Mediterranean range of Chile.pdf  


In [10]:
!unzip "Camel Papers Train.zip"

Archive:  Camel Papers Train.zip
  inflating: Camel Papers Train/Antibody response to the epsilon toxin ofClostridium perfringensfollowing vaccination of Lama glamacrias.pdf  
  inflating: Camel Papers Train/Comparative pigmentation of sheep, goats, and llamas what colors are possible through selection.pdf  
  inflating: Camel Papers Train/Conservative management of a ruptured.pdf  
  inflating: Camel Papers Train/Evaluation of cholesterol and vitamin E concentrations in adult alpacas and nursing crias.pdf  
  inflating: Camel Papers Train/Influence of effects on quality traits and relationships between traits of the llama fleece..pdf  
  inflating: Camel Papers Train/Influence of Follicular Fluid on in Vitro.pdf  
  inflating: Camel Papers Train/Neurological Causes of Diaphragmatic Paralysis in 11 Alpacas.pdf  
  inflating: Camel Papers Train/On the morphology of the cerebellum of the alpaca (Lama pacos)..pdf  
  inflating: Camel Papers Train/Relationships between integumental charact

Now we can begin building our simple index for each of the training directories, and the validation directories.

We will use LlamaIndex's `SimpleNodeParser` to achieve this!

In [11]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

TRAIN_FILES = "Camel Papers Train"
EVAL_FILES = "Camel Papers Test"

In [12]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

def load_corpus(directory, verbose=False):
    if verbose:
        print(f"Loading files in {directory}")

    reader = SimpleDirectoryReader(directory)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [13]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
eval_nodes = load_corpus(EVAL_FILES, verbose=True)

Loading files in Camel Papers Train
Loaded 91 docs


Parsing nodes:   0%|          | 0/91 [00:00<?, ?it/s]

Parsed 156 nodes
Loading files in Camel Papers Test
Loaded 9 docs


Parsing nodes:   0%|          | 0/9 [00:00<?, ?it/s]

Parsed 17 nodes


Now that we've split our source documents into a number of nodes, we can move on to constructing a fine-tuning dataset.

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-3.5-turbo`.

We'll start by using LlamaIndex's `generate_qa_embedding_pairs` and storing it in a `EmbeddingQAFinetuneDataset`.

The basic idea here is straightforward enough:

1. We look at a node
2. We generate a question that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

> NOTE: Keep in mind that the below example uses 100 nodes to generate the QA pairs. This results in 100 calls to `gpt-3.5-turbo` feel free to reduce the number of nodes.

In [14]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [15]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

In [16]:
train_dataset = generate_qa_embedding_pairs(train_nodes[:100], llm=llm)
train_dataset.save_json("train_dataset.json")

100%|██████████| 100/100 [03:05<00:00,  1.86s/it]


In [17]:
eval_dataset = generate_qa_embedding_pairs(eval_nodes[:10], llm=llm)
eval_dataset.save_json("eval_dataset.json")

100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


In [18]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
eval_dataset = EmbeddingQAFinetuneDataset.from_json("eval_dataset.json")

## Task 4: Fine-tuning `snowflake-arctic-embed-l`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-l`](https://huggingface.co/Snowflake/snowflake-arctic-embed-l) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

> NOTE: If you are limited by your compute - you can use the `snowflake-arctic-embed-m` model instead, which will run on the free T4 GPU instance in Colab.

####❓ Question 1:

How many parameters does `snowflake-arctic-embed-l` have?

### Answer
335M parameters

In [19]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset, # Dataset to be trained on
    val_dataset=eval_dataset, # Dataset to evaluate on
    model_id="Snowflake/snowflake-arctic-embed-l", # HuggingFace reference to base embeddings model
    model_output_path="snowflake_finetune_camelids", # Output directory for fine-tuned embeddings model
    epochs=4 # Number of Epochs to train for
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

All that's left to do now is call `.finetune()`!

In [20]:
finetune_engine.finetune()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Now that we've fine-tuned our embeddings model, lets grab the model out of the engine so we can use it later!

> NOTE: You should be able to safely avoid any warnings relating to weights here.

In [21]:
finetuned_embedding_model = finetune_engine.get_finetuned_model()




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from sentence_transformers import SentenceTransformer

fine_tuned_embedding = SentenceTransformer(
    "snowflake_finetune_camelids"
)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
fine_tuned_embedding.push_to_hub(repo_id="prasannab2001/snowflake-ft-camelids-l")

HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-663a9337-12230a7c6728dcb22a378801;9f8c26fd-f318-4ebb-911b-7f5a206e58f5)

You already created this model repo

## Task 5: Evaluating Retrieval with Embedding Model

Now that we've fine-tuned our model - let's see how it performs against OpenAI's `text-embedding-3-small` model, and the base non-fine-tuned version of the model.

In [26]:
from tqdm.notebook import tqdm
from llama_index.core.schema import TextNode
from llama_index.core import Settings, VectorStoreIndex


def evaluate(
    dataset,
    embed_model,
    top_k=2,
    verbose=False,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items() if text != ""]
    index = VectorStoreIndex(
        nodes,
        show_progress=True,
        embed_model=embed_model
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            'is_hit': is_hit,
            'retrieved': retrieved_ids,
            'expected': expected_id,
            'query': query_id,
        }
        eval_results.append(eval_result)
    return eval_results

####❓Question 2:

Describe what the `evaluate` function is doing in the above cell in natural language.

###Answer
The evaluate function calculates whether the retrieval system can find the expected relevant documents for each query. It returns a list of results that can be used to analyze the system's performance. Each result indicates if the expected document was found, the documents that were retrieved, and the expected document's ID.

In [27]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_sentence_transformers(
    dataset,
    model_id,
    name,
):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="/content/")

####❓Question 3:

Describe what the `evaluate_st` function is doing in the above cell in natural language.

###Answers
--Uses the InformationRetrievalEvaluator to evaluate the retrieval performance of a Sentence Transformers model.

--Outputs evaluation results (metrics and detailed performance data) to the specified directory.

In [28]:
import json

with open("eval_dataset.json", 'r+') as f:
    eval_dataset_json = json.load(f)

### Text Embedding 3 Small Results

We'll compare our results against OpenAI's `text-embedding-3-small` model, so we'll need to load it up!

In [29]:
from llama_index.embeddings.openai import OpenAIEmbedding

text_embedding_3_small = OpenAIEmbedding(model="text-embedding-3-small")
te3_val_results = evaluate(eval_dataset_json, text_embedding_3_small)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Let's look at what an example of our results looks like.

In [30]:
import pandas as pd

df_te3 = pd.DataFrame(te3_val_results)

In [31]:
df_te3

,is_hit,retrieved,expected,query
0,True,"[5704bb6d-d346-4d2b-afb4-ef60d8012503, 2cff2db...",5704bb6d-d346-4d2b-afb4-ef60d8012503,cce83a37-7afb-4675-b65a-e2bf37dc2a36
1,True,"[5704bb6d-d346-4d2b-afb4-ef60d8012503, 0be8b07...",5704bb6d-d346-4d2b-afb4-ef60d8012503,8666ee75-7ed7-4dfe-ac08-a5db826038b0
2,True,"[ad63629f-6390-42fc-8920-592d71270c25, 68865ef...",ad63629f-6390-42fc-8920-592d71270c25,32eff09d-edec-40e9-b353-e88b28910062
3,True,"[ad63629f-6390-42fc-8920-592d71270c25, 1f5c674...",ad63629f-6390-42fc-8920-592d71270c25,6a74c39e-32ef-406d-b847-337ef8912408
4,True,"[1f5c674d-f4e1-4373-967a-620f6ca0cab6, 0be8b07...",1f5c674d-f4e1-4373-967a-620f6ca0cab6,8833709a-87ca-4928-afb2-9a4d52af5067
5,True,"[1f5c674d-f4e1-4373-967a-620f6ca0cab6, 0be8b07...",1f5c674d-f4e1-4373-967a-620f6ca0cab6,f7ff16c6-30bd-456c-b15e-dad634fbaaa9
6,True,"[0be8b076-7e07-47a4-9d24-06af683e6569, 2cff2db...",0be8b076-7e07-47a4-9d24-06af683e6569,54e55746-b581-4abb-a8a6-4d4ccede93e6
7,True,"[0be8b076-7e07-47a4-9d24-06af683e6569, 1f5c674...",0be8b076-7e07-47a4-9d24-06af683e6569,01b67780-c1db-483e-82f8-69dca5a09904
8,False,"[68865ef7-5449-43fa-a0d9-3bbb3cff5a02, 83daab6...",2cff2dbe-359a-4ec9-ae2b-2537e064ccc7,d09575bd-6179-4322-9f24-c4556944d74c
9,True,"[2cff2dbe-359a-4ec9-ae2b-2537e064ccc7, 68865ef...",2cff2dbe-359a-4ec9-ae2b-2537e064ccc7,086541ba-8fab-4452-afd3-2b7d9c6cd17e


####❓Question 4:

What do these `[313de41e-534b...]` IDs mean?

###Answer
The UUID-like IDs (e.g., [313de41e-534b...]) are unique identifiers used to represent different queries or documents.
These IDs help to distinguish between the individual queries and documents for accurate evaluation.

Now let's look at the mean value of `is_hit`.

In [32]:
hit_rate_ada = df_te3['is_hit'].mean()
hit_rate_ada

0.95

Overall, we see `text-embedding-3-small` getting a `0.9` "hit rate".

### Base Embeddings Model Results

Let's get the evaluation for our base embedding model (pre-fine-tuning).

In [33]:
base_embed_model_id = "Snowflake/snowflake-arctic-embed-l"
base_embed_model = SentenceTransformer(base_embed_model_id)

arctic_base = "local:Snowflake/snowflake-arctic-embed-l"
arctic_base_val_results = evaluate(eval_dataset_json, arctic_base)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [34]:
df_arctic_base = pd.DataFrame(arctic_base_val_results)

In [35]:
hit_rate_bge = df_arctic_base['is_hit'].mean()
hit_rate_bge

0.5

With a `0.5` hit rate - the base embedding model is absolutely terrible when compared to `text-embedding-3-small` from OpenAI!

Because this is a local `SentenceTransformer`, we can evaluate it with the `SentenceTransformer` evaluation helper-function as well!

In [36]:
evaluate_sentence_transformers(eval_dataset_json, "Snowflake/snowflake-arctic-embed-l", name='arctic-l')

0.5349999999999999

Not great results - let's see what fine-tuning can do for us!

### Fine-tuned Results

In [37]:
finetuned = "local:snowflake_finetune_camelids"
eval_results_finetuned = evaluate(eval_dataset_json, finetuned)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [38]:
df_finetuned = pd.DataFrame(eval_results_finetuned)

In [39]:
hit_rate_finetuned = df_finetuned['is_hit'].mean()
hit_rate_finetuned

0.9

This is a marked improvement when compared to the base model. Absolutely fantastic!

In [40]:
evaluate_sentence_transformers(eval_dataset_json, "snowflake_finetune_camelids", name='finetuned')




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_camelids and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8458333333333334

It's also a marked improvement on the `SentenceTransformer` evaluation!

### Conclusion

Now we can compare the 3 embeddings models to see which performed the best!

In [41]:
df_te3['model'] = 'te3'
df_arctic_base['model'] = 'arctic-baseline'
df_finetuned['model'] = 'arctic-fine-tuned'

In [42]:
df_all = pd.concat([df_te3, df_arctic_base, df_finetuned])
df_all.groupby('model').mean('is_hit')

,is_hit
model,
arctic-baseline,0.50
arctic-fine-tuned,0.90
te3,0.95
